<a href="https://colab.research.google.com/github/AishaEvering/LLM_Mastery/blob/main/GPT_for_Instruction_Following.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 -qqq install datasets
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import(
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [ ]:
def preprocess(example):
  example['prompt'] = f"{example['instruction']} {example['input']} {example['output']}"

  return example

In [ ]:
def tokenize_dataset(dataset):
  tokenized_dataset = dataset.map(lambda example: tokenizer(example['prompt'], truncation=True, max_length=128), batched=True, remove_columns=['prompt'])
  return tokenized_dataset

In [ ]:
dataset = load_dataset('hakurei/open-instruct-v1', split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/498813 [00:00<?, ? examples/s]

In [ ]:
dataset.to_pandas().sample(20)

,output,instruction,input
331413,"You have a lot of experience in the field, and...",Explain why this job candidate would be good f...,
467041,The capital city of Japan is Tokyo.,What is the capital city of Japan.,
467717,The process for obtaining a real estate licens...,I'm interested in getting a real estate licens...,
47087,A comma should be used when listing three or m...,When can a comma be used with a list of three ...,
44610,The three countries in the Americas are the Un...,List three countries in the Americas.,
104110,70,Give me a single number between 0 and 100 that...,
396456,an,Find the correct word in the given set of word...,
215008,"Yes, tempeh is a plant based food",Is tempeh a vegetarian food?,
423864,To create an alert in Oracle Cloud if your dai...,how can I create an alert in oracle cloud if m...,
69171,"Daughters: Alice, Betty, Carol\nSolution: The ...",Describe a way to solve the following problem:...,


In [ ]:
formatted_dataset = dataset.map(preprocess, remove_columns=['instruction', 'input', 'output'])

Map:   0%|          | 0/498813 [00:00<?, ? examples/s]

In [ ]:
ds_sample = formatted_dataset.shuffle(seed=42).select(range(100000))

ds = ds_sample.train_test_split(test_size=0.1)
ds

DatasetDict({
    train: Dataset({
        features: ['prompt'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['prompt'],
        num_rows: 10000
    })
})

In [ ]:
ds['train'][0]

{'prompt': 'Construct a Python program to print the numbers from 1 to 10 on a single line.\n  for i in range(1,11):\n  print(i, end=" ")'}

In [ ]:
train_ds = ds['train']
test_ds = ds['test']

train_ds, test_ds

(Dataset({
     features: ['prompt'],
     num_rows: 90000
 }),
 Dataset({
     features: ['prompt'],
     num_rows: 10000
 }))

In [ ]:
MODEL_NAME = 'microsoft/DialoGPT-medium'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
train_dataset = tokenize_dataset(train_ds)
test_dataset = tokenize_dataset(test_ds)

Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 90000
})

In [ ]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm=False)

In [ ]:
training_args = TrainingArguments(
    output_dir='experiments',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
)

In [ ]:
# training pipeline
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    data_collator = data_collator,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,1.740800
1000,1.734200
1500,1.701600
2000,1.686700
2500,1.660000
3000,1.666500
3500,1.665400
4000,1.643000
4500,1.638400
5000,1.633400


TrainOutput(global_step=11250, training_loss=1.7632461968315971, metrics={'train_runtime': 2480.9417, 'train_samples_per_second': 36.277, 'train_steps_per_second': 4.535, 'total_flos': 2.061011760857088e+16, 'train_loss': 1.7632461968315971, 'epoch': 1.0})

In [ ]:
prompt = ''

In [ ]:
def generate_text(prompt):
  inputs = tokenizer.encode(prompt, return_tensors='pt').to('cuda')
  outputs = model.generate(inputs, max_length=64, pad_token_id=tokenizer.eos_token_id)
  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  return generated_text[:generated_text.rfind('.') + 1]

In [ ]:
generate_text("What's the best way to cook chicken breast?")

"What's the best way to cook chicken breast?  - Heat the chicken breast in a pan.\n- Add salt and pepper to taste.\n- Add oil to taste.\n- Add chicken to pan.\n- Add flour to taste.\n- Add oil to pan.\n- Add chicken to pan."

In [ ]:
generate_text("Should i invest in stocks?")

'Should i invest in stocks?  It depends on your goals and risk tolerance. It is important to understand the risks associated with investing in stocks and the potential for returns. It is also important to understand the risks associated with investing in other types of investments, such as bonds, mutual funds, and ETFs.'

In [ ]:
generate_text("I need a place to go for this summer vacation, what locations would you recommend.")

'I need a place to go for this summer vacation, what locations would you recommend.'

In [ ]:
generate_text("What's the fastest route from NY City to Boston?")

"What's the fastest route from NY City to Boston?  The fastest route from New York City to Boston is via the Red Line, which takes about 3 hours. The quickest route is via the Blue Line, which takes about 2.5 hours. The fastest route is via the Yellow Line, which takes about 2."